In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from utilities import plot_confusion_matrix,plot_roc_auc,plot_classification_report
from utilities import grid_search_simple,graph_result,get_classes
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import json

In [2]:
full_df = pd.read_csv("..\..\data\indeed.pre_processing.csv")

In [3]:
for col in full_df.columns:
    print(col)

Unnamed: 0.1
_id
adresse
date_de_publication
description
localisation
nom_entreprise
salaire
titre
type_de_contrat
url
Lyon
niveau_etude
type_de_cursus
type_contrat_description
grande_categorie
type_contrat_titre
Nantes
Paris
Toulouse
salaire_moyen
salaire_min
salaire_max
bac + 1
bac + 2
bac + 3
bac + 4
bac + 5
bac + 8
diplôme supérieur
ingénieur
master 1
master 2
autodidacte
ecole ingénieur
grande école
ingénieur informatique
école de commerce
école ingénieur
Apprentissage
Autre
CDD
CDI
Contrat pro
Freelance
Indépendant
Intérim
Stage
Temps partiel
Temps plein
alternance_titre
cdd_titre
cdi_titre
freelance_titre
stage_titre
3 mois renouvelable_desc
3mois renouvelable_desc
alternance_desc
cdd_desc
cdi_desc
freelance_desc
stage_desc
back-end
développeu web
développeur data
développeur mobile
développeur web
front-end
abcl
ada
afnix
agora
alf
algol
apl
applescript
asp.net
autoit
awk
back end
bash
basic
beanshell
beta
bliss
c
c#
c\+\+
cecil
charity
chuck
cilk
clean
cleo
clist
cobol
cobra
c

In [4]:
df = full_df.copy()
cols_to_drop = ["Unnamed: 0.1","adresse","date_de_publication","description","localisation","nom_entreprise","salaire",
        "titre","type_de_contrat","url","niveau_etude","type_de_cursus","type_contrat_description","grande_categorie", "type_contrat_titre"
               ,'salaire_min','salaire_max','3 mois renouvelable_desc','3mois renouvelable_desc']
df.drop(cols_to_drop, axis=1, inplace=True)

In [5]:
y = df['salaire_moyen'][pd.notnull(df['salaire_moyen'])]
X = df.drop(columns = ['salaire_moyen',"_id"])
X = X[pd.notnull(df['salaire_moyen'])]

In [6]:
y_class, dic_classe_labels,dic_min_max_mean = get_classes(y)

In [7]:
svm = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.25, random_state=0)

In [9]:
svm.fit(X_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [10]:
y_pred_reglog = svm.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred_reglog)

0.706199460916442

In [12]:
df_predicted = df.drop(columns = ['salaire_moyen'])
df_predicted.reset_index(inplace=True, drop=True)
X_df_predicted = df_predicted.drop(columns = ['_id'])
X_df_predicted.reset_index(inplace=True, drop=True)

In [13]:
#%%time
y_pred_w_s = svm.predict(X_df_predicted)

In [14]:
salaire_moyen = []
salaire_min = []
salaire_max = []
for i in y_pred_w_s:
    salaire_min.append(dic_min_max_mean[i][0])
    salaire_moyen.append(dic_min_max_mean[i][1])
    salaire_max.append(dic_min_max_mean[i][2])

In [15]:
df_predicted['salaire_moyen'] = pd.DataFrame(salaire_moyen)
df_predicted['salaire_min'] = pd.DataFrame(salaire_min)
df_predicted['salaire_max'] = pd.DataFrame(salaire_max)

In [16]:
cols =["adresse","type_de_contrat","niveau_etude", "localisation","type_de_cursus"]
df_predicted[cols] = full_df[pd.isna(full_df["salaire_moyen"])][cols]

In [17]:
cols.append("salaire_moyen")
cols.append("salaire_min")
cols.append("salaire_max")
#df_predicted = df_predicted[cols]
df_predicted

,_id,Lyon,Nantes,Paris,Toulouse,bac + 1,bac + 2,bac + 3,bac + 4,bac + 5,...,zipkin,centre,salaire_moyen,salaire_min,salaire_max,adresse,type_de_contrat,niveau_etude,localisation,type_de_cursus
0,5da04bf5559bce5541bcce8f,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,26000,46000,Toulouse (31),NaN,ingénieur,Toulouse,NaN
1,5da04bf5559bce5541bcce90,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,26000,46000,Toulouse (31),CDI,NaN,Toulouse,NaN
2,5da04bf5559bce5541bcce91,0,0,0,1,0,0,0,0,0,...,0,0,36000.0,26000,46000,Labège (31),CDI,ingénieur,Toulouse,NaN
3,5da04bf5559bce5541bcce92,0,0,0,1,0,0,0,0,0,...,0,1,66500.0,46000,87000,Toulouse (31),CDI,ingénieur,Toulouse,autodidacte
4,5da04bf5559bce5541bcce93,0,0,0,1,0,0,0,0,0,...,0,1,36000.0,26000,46000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8010,5dad7c638798f0c2b8c4329e,0,0,1,0,0,0,0,0,1,...,0,1,13000.0,0,26000,Paris (75),Stage,bac + 5,Paris,NaN
8011,5dad7cc28798f0c2b8c432ff,1,0,0,0,0,0,0,0,0,...,0,1,36000.0,26000,46000,NaN,NaN,NaN,NaN,NaN
8012,5dad7d028798f0c2b8c4333a,1,0,0,0,0,1,0,0,0,...,0,1,36000.0,26000,46000,Lyon (69),"Temps plein, CDI",bac + 2,Lyon,NaN
8013,5dad7d038798f0c2b8c4333b,1,0,0,0,0,0,0,0,0,...,0,1,36000.0,26000,46000,NaN,NaN,NaN,NaN,NaN


In [18]:
df_predicted.to_csv("..\..\data\indeed.predicted.csv")

In [19]:
xx =['tableau', 'powerbi','symfony', 'jquery','angular','react','react native','node js','git','github','AWS'
            'visual studio','django','flask','api rest','laravel', 'hololens', 'docker', 'jira', 'scrum', 'kanban', 
               'azure','aws', 'teamcity', 'jenkins',"oracle", "mysql","Microsoft SQL Server", "PostgreSQL","DB2","Microsoft Access",
                "SQLite","Teradata", "SAP Adaptative Server", "Hive", "FileMaker"
             ,"SAP HANA", "MariaDB", "Informix", "Firebird", "microsoft azure sql database", "Vertica", "Netezza", "Ingres", "Greenplum", "mongodb","Liquibase",
                "Buddy", "Semaphore", "GoCD", "Drone.io", "TeamCity", "Wercker", "Codeship", "Travis CI", "CircleCI", "Bamboo","Jenkins",
                "VersionOne", "Phabricator", "Asana", "Pivotal Tracker", "Basecamp", "Visual Studio", "Trello", "Jira Software", "Mingle",
                "Kallithea", "Beanstalk", "GitBucket", "Mercurial", "Gogs", "GitLab", "Bitbucket", "GitHub", "Subversion", "Git",
                "SoapUI", "Katalon Studio", "ThreatModeler", "Checkmarx", "RSpec", "SpecFlow", "Pa11y", "Browsersync",
                "Serverspec", "pytest", "BlazeMeter", "Load Impact", "Galen Framework",
                "TestNG", "QUnit", "NUnit", "FitNesse", "Karma", "Gatling", "OWASP ZAP", "Gauntlt", "Mocha", "Jmeter",
                "Cucumber", "JUnit", "Jasmine", "Selenium","Nagios","Keen IO","Opsgenie","Beats","Moogsoft","PagerDuty","Rollbar","Raygun",
                "Graphite","Grafana","APImetrics","Riemann","Atlas","Runscope","Dynatrace","Sensu","Pinpoint","Prometheus"
                ,"Vizceral","Sentry","Google Analytics","Grok","Zipkin","Zabbix","Datadog","Kibana","Elasticsearch","Logstash","Airbrake","New Relic","App Dynamics","Vector","Splunk",
                "Morpheus", "Dokku", "Engine Yard", "OpenShift", "Cloud Foundry", "Flynn", "Azure", "OpenStack",
                "Rackspace", "Google Cloud Platform", "Heroku", "Amazon Web Services","sql server"
                ]

In [4]:
import numpy as np
xx = [i.lower() for i in xx]
xx = np.unique(xx)
xx

array(['airbrake', 'amazon web services', 'angular', 'api rest',
       'apimetrics', 'app dynamics', 'asana', 'atlas', 'aws',
       'awsvisual studio', 'azure', 'bamboo', 'basecamp', 'beanstalk',
       'beats', 'bitbucket', 'blazemeter', 'browsersync', 'buddy',
       'checkmarx', 'circleci', 'cloud foundry', 'codeship', 'cucumber',
       'datadog', 'db2', 'django', 'docker', 'dokku', 'drone.io',
       'dynatrace', 'elasticsearch', 'engine yard', 'filemaker',
       'firebird', 'fitnesse', 'flask', 'flynn', 'galen framework',
       'gatling', 'gauntlt', 'git', 'gitbucket', 'github', 'gitlab',
       'gocd', 'gogs', 'google analytics', 'google cloud platform',
       'grafana', 'graphite', 'greenplum', 'grok', 'heroku', 'hive',
       'hololens', 'informix', 'ingres', 'jasmine', 'jenkins', 'jira',
       'jira software', 'jmeter', 'jquery', 'junit', 'kallithea',
       'kanban', 'karma', 'katalon studio', 'keen io', 'kibana',
       'laravel', 'liquibase', 'load impact', 'logstash

In [6]:
import pandas as pd
tt = pd.read_csv("..\..\data\indeed.pre_processing.csv")
tt.columns

Index(['Unnamed: 0.1', '_id', 'adresse', 'date_de_publication', 'description',
       'localisation', 'nom_entreprise', 'salaire', 'titre', 'type_de_contrat',
       ...
       'trello', 'vector', 'versionone', 'vertica', 'visual studio',
       'vizceral', 'wercker', 'zabbix', 'zipkin', 'centre'],
      dtype='object', length=353)

In [52]:
import pandas as pd

df = pd.read_csv("../../data/indeed.predicted.csv")
locations = np.unique(df[pd.notnull(df["localisation"])]["localisation"]).tolist()
list_depts = {
       "Paris": ["78", "77", "91", "60", "27", "76", "45", "61", "02", "75"],
       "Lyon": ["71", "42", "38", "03", "39", "01", "63", "38", "73", "21", "69"],
       "Toulouse": ["09", "32", "30", "27", "26", "25", "23", "20", "19", "18","17","11","12","13","15","16", "31"],
       "Bordeaux": ["17", "16", "24", "87", "33"],
       "Nantes": ["56", "35", "53", "85", "49", "22", "29", "29", "44"]
   }

result = {}
for index, item in enumerate(locations):
    temp = df[df["localisation"] == item]
    for i, item2 in temp.iterrows():
        address = item2["adresse"]
            
        if item in address:
            address = item
        if item not in result:
            result[item] = {}
        if address not in result[item]:
            result[item][address] = item2["salaire_moyen"]
        else:
            result[item][address] = (result[item][address] + item2["salaire_moyen"]) / 2
print(json.dumps(result["Bordeaux"]))

{"Nouvelle-Aquitaine": 42872.5398555398, "Bordeaux": 36000.001350945684, "33327 Eysines": 36000.0, "Cestas (33)": 37667.96875, "M\u00e9rignac (33)": 36029.8433303833, "33185 Le Haillan": 36000.0, "Pessac (33)": 44019.6533203125, "Can\u00e9jan (33)": 24500.0, "33600 Pessac": 25306.21337890625, "B\u00e8gles (33)": 38859.375, "Gironde": 29182.754516601562, "Le Haillan (33)": 36000.0, "33130 B\u00e8gles": 24500.0, "Floirac (33)": 36000.0, "Le Bouscat (33)": 36000.0, "33650 Martillac": 36000.0, "33700 M\u00e9rignac": 36000.0, "France": 36000.0, "33610 Can\u00e9jan": 24500.0, "Saint-Aubin-de-M\u00e9doc (33)": 36000.0, "33520 Bruges": 36000.0, "Bassens (33)": 36000.0, "Lormont (33)": 36000.0, "Eysines (33)": 58875.0, "Bill\u00e8re (64)": 36000.0, "Gradignan (33)": 39750.0, "Villenave-d'Ornon (33)": 66500.0, "Cenon (33)": 13000.0, "34000 Montpellier": 36000.0, "37200 Tours": 66500.0}


In [53]:
print(json.dumps(result["Bordeaux"]))

{"Nouvelle-Aquitaine": 42872.5398555398, "Bordeaux": 36000.001350945684, "33327 Eysines": 36000.0, "Cestas (33)": 37667.96875, "M\u00e9rignac (33)": 36029.8433303833, "33185 Le Haillan": 36000.0, "Pessac (33)": 44019.6533203125, "Can\u00e9jan (33)": 24500.0, "33600 Pessac": 25306.21337890625, "B\u00e8gles (33)": 38859.375, "Gironde": 29182.754516601562, "Le Haillan (33)": 36000.0, "33130 B\u00e8gles": 24500.0, "Floirac (33)": 36000.0, "Le Bouscat (33)": 36000.0, "33650 Martillac": 36000.0, "33700 M\u00e9rignac": 36000.0, "France": 36000.0, "33610 Can\u00e9jan": 24500.0, "Saint-Aubin-de-M\u00e9doc (33)": 36000.0, "33520 Bruges": 36000.0, "Bassens (33)": 36000.0, "Lormont (33)": 36000.0, "Eysines (33)": 58875.0, "Bill\u00e8re (64)": 36000.0, "Gradignan (33)": 39750.0, "Villenave-d'Ornon (33)": 66500.0, "Cenon (33)": 13000.0, "34000 Montpellier": 36000.0, "37200 Tours": 66500.0}


In [14]:
preprocessing_df = pd.read_csv("..\..\data\indeed_mongo.csv")

In [15]:
preprocessing_df.shape

(8015, 12)

In [19]:
preprocessing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8015 entries, 0 to 8014
Data columns (total 12 columns):
Unnamed: 0             8015 non-null int64
Unnamed: 0.1           6756 non-null float64
_id                    8015 non-null object
adresse                8015 non-null object
date_de_publication    8004 non-null object
description            8015 non-null object
localisation           8015 non-null object
nom_entreprise         8009 non-null object
salaire                1484 non-null object
titre                  8015 non-null object
type_de_contrat        5338 non-null object
url                    8015 non-null object
dtypes: float64(1), int64(1), object(10)
memory usage: 751.5+ KB


In [18]:
(preprocessing_df.isna().sum()*100)/len(preprocessing_df)

Unnamed: 0              0.000000
Unnamed: 0.1           15.708047
_id                     0.000000
adresse                 0.000000
date_de_publication     0.137243
description             0.000000
localisation            0.000000
nom_entreprise          0.074860
salaire                81.484716
titre                   0.000000
type_de_contrat        33.399875
url                     0.000000
dtype: float64

In [9]:
percent_missing = preprocessing_df.isnull().sum() * 100 / len(preprocessing_df)
missing_value_df = pd.DataFrame({'column_name': preprocessing_df.columns,
                                 'percent_missing': percent_missing})

In [13]:
i = 0
for index, item in missing_value_df.iterrows():
    if item["percent_missing"] < 30:
       i = i +  1
print(i)

344
